# CS 440 Final Project: Part 1

## Enhancing Connect-4: A Comparative Analysis of MCTS, Neural Networks, and Advanced Reinforcement Learning Techniques

---

Sofia Catalan, Quincy Meisman, and Thomas Stewart \
Department of Computer Science, Colorado State University \
CS 440: Introduction to Artificial Intelligence \
Dr. Fabio Santos 

## Table of Contents

- Monte Carlo Tree Search (MCTS):
    - Overview
    - Code
    - Testing
        - MCTS vs. Human
        - MCTS vs. Random Player
        - MCTS vs. MCTS
- References
- Conclusion

In [1]:
from notebook import psource
from mcts import *
from connect4 import *
from players import *

## Monte Carlo Tree Search (MCTS): Overview

As mentioned in our research paper, we decided to use MCTS as our baseline model for Connect 4. To implement this, we developed three .py files, each with its specific purpose:

* **mcts.py**:
    * Implements the four phases of the Monte Carlo Tree Search: Selection, Expansion, Simulation/Rollout, and Back-propagation
    * Defines a node in the MCTS, storing the state of the game (Connect 4), information about the parent and children nodes, etc.
* **connect4.py**: Models the game of Connect 4 for simulation purposes, handling player turns, generation of legal moves, game-over condition checking, and game outcome.
* **players.py**: Defines players for Connect 4 against which MCTS can be tested.

Additionally, we used **notebook.py** for visualization purposes, borrowed from [https://github.com/aimacode/aima-python](https://github.com/aimacode/aima-python)

While this code is our own implementation of the algorithm, we were significantly inspired by the following sources:

* The implementation from [Olin College](https://mcts.netlify.app/)
* The implementation from [Qi Wang](https://www.harrycodes.com/blog/monte-carlo-tree-search), including his YouTube video *[Beating Connect 4 with Monte Carlo Tree Search! | Explanation + Code](https://youtu.be/EB-NJtNERBQ?si=s4_6m3WHpjcqionN)* explaining his approach
* The following MCTS flowchart obtained from John Levine in his YouTube video *[Monte Carlo Tree Search](https://youtu.be/UXW2yZndl7U?si=X8SAZcUz4DNxRcVr)*:



In [2]:
from IPython.display import Image, HTML, display

html_code = """
<div style="text-align: center;">
    <img src="mcts_flowchart.jpg" style="width: 400px;">
</div>
"""
display(HTML(html_code))

## Monte Carlo Tree Search: Code

In **mcts.py**, the four phases of MCTS are implemented:

### Selection

In the first phase of MCTS, the tree is traversed starting from the root node, using the UCB1 formula to select child nodes, and continuing until a leaf node is reached.

In [3]:
psource(MCTS.select)

### Expansion

When a leaf node is reached, the goal of the Expansion phase is to determine whether the tree can be expanded. The algorithm does this by checking for the following three cases:

* **If the leaf node has no valid moves left (i.e., no children)**: expansion is skipped and the node is returned for a simulation in the Simulation/Rollout Phase, which concludes right away.
* **If the leaf node has not been in a simulation before (i.e., N = 0) and it is not the root**: expansion is skipped and the node is returned so that a simulation can be performed on it in the Simulation/Rollout Phase.
* **If the leaf node is the root or has been in a simulation before**: the tree is expanded by generating all possible child nodes, and one of these children is selected for the Simulation/Rollout Phase. The child chosen can either be the first child or a random child, and following the flowchart above, we will select the first child.

In [4]:
psource(MCTS.expand)

### Simulation/Rollout

In this phase, a playout is carried out starting from the selected child node and continues until the game is over, either by a win or a draw. The outcome is then returned so that the tree can be updated in the Back-propagation phase. 

In [5]:
psource(MCTS.simulate)

### Back-propagation

The following updates occur for each node along the path up to the root node:
* **Number of Simulations (N)**: The number of simulations is incremented.
* **Wins (U)**: The number of wins is updated, rewarding the players based on the outcome of the simulation (1 for a win, 0 for a loss or a draw).
* **UCB1**: The UCT value is updated to reflect the new exploration-exploitation balance for that node.

In [6]:
psource(MCTS.backpropagate)

## Monte Carlo Tree Search: Testing

In the following section, the MCTS algorithm will be tested by playing against a Human Player, a Random Player, and another instance of MCTS.

Playing against a Human will help provide a preliminary glimpse into the decision-making process of the MCTS algorithm.

Even though a Random Player is not the most insightful way to evaluate the MCTS algorithm—as it lacks strategy in its gameplay—it still serves as a useful baseline to assess if the algorithm actually works.

The MCTS algorithm will also be tested against another instance of itself, with each having different time limits for the search. This will help us evaluate how the algorithm performs differently when given different time constraints to find the optimal move.

### Testing Loop

In [7]:
def playConnect4(playerXType, playerOType, numGames=100, timeLimitX=1, timeLimitO=1, verbose=True):
    playerX = None 
    playerO = None
    playerXWins = 0
    playerOWins = 0
    draws = 0
    totalMoves = 0

    # Iterate for x amount of games
    for i in range(numGames):
        # Create new Connect 4 game
        game = Connect4()

        # Initialize players one and two
        if playerXType == "MCTS":
            playerX = MCTS(gameRoot=deepcopy(game))
        elif playerXType == "Random":
            playerX = RandomPlayer()
        else:
            playerX = HumanPlayer()

        if playerOType == "MCTS":
            playerO = MCTS(gameRoot=deepcopy(game))
        elif playerOType == "Random":
            playerO = RandomPlayer()
        else:
            playerO = HumanPlayer()

        # Initial state of the game
        if verbose:
            print("\n************************************")
            print(f"      Starting Game {i + 1} of       ")
            print("           Connect 4!              ")
            print("************************************")
            print(game)

        while not game.isGameOver():
            if verbose:
                print(f"Player {game.player}'s turn:")
            if game.player == PIECE_ONE: # Player x's turn to choose a move
                if isinstance(playerX, MCTS):
                    if verbose:
                        print("Searching for the best move using MCTS...")
                    numSimulations = playerX.search(timeLimitX)
                    if verbose:
                        print(f"Search complete. Total simulations: {numSimulations}")
                    move = playerX.getBestMove()
                elif isinstance(playerX, RandomPlayer):
                    move = playerX.randomMove(game)
                else: 
                    move = playerX.move(game)
            else: # Player o's turn to choose a move
                if isinstance(playerO, MCTS):
                    if verbose:
                        print("Searching for the best move using MCTS...")
                    numSimulations = playerO.search(timeLimitO)
                    if verbose:
                        print(f"Search complete. Total simulations: {numSimulations}")
                    move = playerO.getBestMove()
                elif isinstance(playerO, RandomPlayer):
                    move = playerO.randomMove(game)
                else:
                    move = playerO.move(game)
            if verbose:
                print(f"Player {game.player} chose to drop their piece in column: {move}")
            # Apply move to board and print out the changes
            game.dropPiece(move)
            if verbose:
                print(game)
            
            # Update root in MCTS to reflect current game
            if isinstance(playerX, MCTS):
                playerX.root = Node(parent=None, move=None, gameState=deepcopy(game))
            if isinstance(playerO, MCTS):
                playerO.root = Node(parent=None, move=None, gameState=deepcopy(game))  

            totalMoves += 1
            game.switchPlayer()
            
        if verbose:
            print("Game is Over!")
        outcome = game.getOutcome()

        # The winner corresponds to the player who made the last winning move.
        # since game.isGameOver() is checked after switchPlayer() is called,
        # getOutcome() reflects the player whose turn it is to play now, not the winner.
        # So, the winner is the opposite of the outcome.
        if outcome == OUTCOME_ONE:
            playerOWins += 1
            winner = 'o'
        elif outcome == OUTCOME_TWO:
            playerXWins += 1
            winner = 'x'
        else:
            draws += 1
            winner = "Draw"
            
        if verbose:
            print(f"The winner of Game {i + 1} is: {winner}!")     
    
    # Calculate win percentages of each player, draw percentages, and the average 
    # number of moves performed during each game
    playerXWinPercentage = (playerXWins / numGames) * 100
    playerOWinPercentage = (playerOWins / numGames) * 100
    drawPercentage = (draws / numGames) * 100
    avgMoves = totalMoves / numGames
            
    return playerXWinPercentage, playerOWinPercentage, drawPercentage, avgMoves

## MCTS vs. Human

In the following example, a human player competed against our MCTS. The MCTS was given 1 second to search for the best move, and as shown, the human won this time. What if the MCTS was given more time to search?

In [8]:
pXType = "MCTS"
pOType = "Human"
pXWinPercentage, pOWinPercentage, drawPercentage, totalMoves = playConnect4(pXType, pOType, 1)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Total Moves: {totalMoves} moves")


************************************
      Starting Game 1 of       
           Connect 4!              
************************************
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 5183
Player x chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | | |
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  3


Player o chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | | |x| | | |
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 5305
Player x chose to drop their piece in column: 4
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | | |x|x| | |
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  2


Player o chose to drop their piece in column: 2
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | |o|x|x| | |
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 5742
Player x chose to drop their piece in column: 5
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | |o|x|x|x| |
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


...

Player o chose to drop their piece in column: 4
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | |o| | |
| | | |o|x| | |
| | |o|x|x|x|o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 6111
Player x chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x|o| | |
| | | |o|x| | |
| | |o|x|x|x|o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  5


Player o chose to drop their piece in column: 5
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x|o| | |
| | | |o|x|o| |
| | |o|x|x|x|o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 6349
Player x chose to drop their piece in column: 4
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | |x| | |
| | | |x|o| | |
| | | |o|x|o| |
| | |o|x|x|x|o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  3


Player o chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | |o|x| | |
| | | |x|o| | |
| | | |o|x|o| |
| | |o|x|x|x|o|
---------------
Game is Over!
The winner of Game 1 is: o!
Player X Win Percentage: 0.0%
Player O Win Percentage: 100.0%
Draw Percentage: 0.0%
Total Moves: 12.0 moves


## MCTS vs. Human: Rematch

This time around, the MCTS was given 10 seconds to search for the best move, and it emerged as the victor!

In [17]:
pXType = "MCTS"
pOType = "Human"
pXWinPercentage, pOWinPercentage, drawPercentage, totalMoves = playConnect4(pXType, pOType, 1, 10)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Total Moves: {totalMoves} moves")


************************************
      Starting Game 1 of       
           Connect 4!              
************************************
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 46240
Player x chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | | |
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  6


Player o chose to drop their piece in column: 6
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | |o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 47506
Player x chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | | |x| | |o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  4


Player o chose to drop their piece in column: 4
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | | |x|o| |o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 49600
Player x chose to drop their piece in column: 3
 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | | |x| | | |
| | | |x|o| |o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


...

Player o chose to drop their piece in column: 2
 0 1 2 3 4 5 6
| | | | | | | |
| | | |o| | | |
| | | |o| | | |
| |x|o|x| | | |
|o|o|x|x|x| | |
|x|x|o|x|o| |o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 62437
Player x chose to drop their piece in column: 2
 0 1 2 3 4 5 6
| | | | | | | |
| | | |o| | | |
| | |x|o| | | |
| |x|o|x| | | |
|o|o|x|x|x| | |
|x|x|o|x|o| |o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  2


Player o chose to drop their piece in column: 2
 0 1 2 3 4 5 6
| | | | | | | |
| | |o|o| | | |
| | |x|o| | | |
| |x|o|x| | | |
|o|o|x|x|x| | |
|x|x|o|x|o| |o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 65461
Player x chose to drop their piece in column: 4
 0 1 2 3 4 5 6
| | | | | | | |
| | |o|o| | | |
| | |x|o| | | |
| |x|o|x|x| | |
|o|o|x|x|x| | |
|x|x|o|x|o| |o|
---------------
Player o's turn:
Legal moves: [0, 1, 2, 3, 4, 5, 6]


Enter a column number to drop your piece:  6


Player o chose to drop their piece in column: 6
 0 1 2 3 4 5 6
| | | | | | | |
| | |o|o| | | |
| | |x|o| | | |
| |x|o|x|x| | |
|o|o|x|x|x| |o|
|x|x|o|x|o| |o|
---------------
Player x's turn:
Searching for the best move using MCTS...
Search complete. Total simulations: 67161
Player x chose to drop their piece in column: 5
 0 1 2 3 4 5 6
| | | | | | | |
| | |o|o| | | |
| | |x|o| | | |
| |x|o|x|x| | |
|o|o|x|x|x| |o|
|x|x|o|x|o|x|o|
---------------
Game is Over!
The winner of Game 1 is: x!
Player X Win Percentage: 100.0%
Player O Win Percentage: 0.0%
Draw Percentage: 0.0%
Total Moves: 21.0 moves


### Try It!

In [ ]:
pXType = "MCTS"
pOType = "Human"

searchTime = 5 # Adjust search times for the MCTS

pXWinPercentage, pOWinPercentage, drawPercentage, totalMoves = playConnect4(pXType, pOType, 1, searchTime)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Total Moves: {totalMoves} moves")

## MCTS vs. Random Player

Next, we pitted our MCTS search algorithm against a Random Player in 1000 games to evaluate its performance. As can be seen, it won 99.5% of the games played, and lost 0.5%, making ~9 moves in a game. This is to be expected, as the Random Player does not rely on any educated decision-making process, but instead makes purely random decisions. On the other hand, the MCTS algorithm relies on the outcome of simulations to inform its move selection process.

In [19]:
pXType = "MCTS"
pOType = "Random"

numGames = 1000

pXWinPercentage, pOWinPercentage, drawPercentage, avgMoves = playConnect4(pXType, pOType, numGames, verbose=False)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Avg. Moves Taken: {avgMoves:.3f} moves")

Player X Win Percentage: 99.5%
Player O Win Percentage: 0.5%
Draw Percentage: 0.0%
Avg. Moves Taken: 8.889 moves


## MCTS vs. MCTS

Lastly, we evaluated the MCTS against itself for 100 games. 

In the first cell, we gave one MCTS 0.0001 seconds, and the other MCTS 5 seconds. Our initial hypothesis was that the more time we give the MCTS, the better it will perform, and by giving them such contrasting time differences, we thought the MCTS with 5 seconds would win a striking majority of the games. However, it only won ~55% of them, and even though that is still the majority of the games, it was not what we were expecting. 

Because of this, we decided to test our MCTS again. This time around, we gave one MCTS instance 3 seconds to come up with the best move, and we gave the other MCTS instance 5 seconds. To our surprise, the MCTS with 3 seconds won 100% of the games.

This may be due to several factors. The most obvious one we could think of was that the more time an MCTS is given to search for the best move, the more time it has to explore suboptimal branches due to the exploration-exploitation tradeoff from the UCT selection policy. So, the more time the MCTS has, the more it explores suboptimal moves, while the less time it has, the more it exploits promising moves.

Another reason may be because we created a simple version of MCTS where the simulation phase does not use a playout policy to guide decisions. Instead, the MCTS does random play during simulations. Because of this, the more time the MCTS is given, the more simulations it can perform, which means it is averaging over potentially noisy simulations that dilute the signal for optimal moves.

In [36]:
pXType = "MCTS"
pOType = "MCTS"

numGames = 100

pXWinPercentage, pOWinPercentage, drawPercentage, avgMoves = playConnect4(pXType, pOType, numGames, timeLimitX=0.0001, timeLimitO=5, verbose=False)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Avg. Moves Taken: {avgMoves:.3f} moves")

Player X Win Percentage: 44.0%
Player O Win Percentage: 55.00000000000001%
Draw Percentage: 1.0%
Avg. Moves Taken: 11.880 moves


In [39]:
pXType = "MCTS"
pOType = "MCTS"

numGames = 100

pXWinPercentage, pOWinPercentage, drawPercentage, avgMoves = playConnect4(pXType, pOType, numGames, timeLimitX=3, timeLimitO=5, verbose=False)

print(f"Player X Win Percentage: {pXWinPercentage}%")
print(f"Player O Win Percentage: {pOWinPercentage}%")
print(f"Draw Percentage: {drawPercentage}%")
print(f"Avg. Moves Taken: {avgMoves:.3f} moves")

Player X Win Percentage: 100.0%
Player O Win Percentage: 0.0%
Draw Percentage: 0.0%
Avg. Moves Taken: 7.000 moves


## Conclusion

In this notebook, we presented our own implementation of a reinforcement learning-based approach to play Connect 4. Through the development of the MCTS algorithm, we learned to leverage an RL method that does not rely on a heuristic evaluation function. Instead, we learned how to use a simulation-based approach to make decisions. Even though some outcomes were expected, others were not, giving us a reason to continue developing our algorithm to understand it better and improve it in the future. One potential improvement we have planned is to add a playout policy to guide the decision-making process during simulations. That way, the simulations can provide more useful information by focusing toward favorable moves rather than random play. We also plan to create a better opponent for MCTS, such as testing it against the Minimax algorithm to see how it compares.


## <center>References</center>

aima-python. (2021). *aima-python* \[Software\]. GitHub. [https://github.com/aimacode/aima-python](https://github.com/aimacode/aima-python)

Ku, M., Daitzman, S., Brehm, D. (2020). *Automated Four in a Row*. Olin College. [https://mcts.netlify.app/](https://mcts.netlify.app/).

Levine, J. (2017). *Monte Carlo Tree Search* \[Video\]. YouTube. [https://youtu.be/UXW2yZndl7U?si=C0hqXtSibaH_In1Y](https://youtu.be/UXW2yZndl7U?si=C0hqXtSibaH_In1Y). 

Wang, Q. (2022a). *Connect 4 with Monte Carlo Tree Search*. Retrieved from [https://www.harrycodes.com/blog/monte-carlo-tree-search](https://www.harrycodes.com/blog/monte-carlo-tree-search).

Wang, Q. (2022b). *Beating Connect 4 with Monte Carlo Tree Search! | Explanation + Code* \[Video\]. YouTube. [https://youtu.be/EB-NJtNERBQ?si=RF0Vu5EfzTi7XTV6](https://youtu.be/EB-NJtNERBQ?si=RF0Vu5EfzTi7XTV6).